<a href="https://colab.research.google.com/github/Neelu2647/Mini-project/blob/maithili_branch/Chest_X_Ray_Images_(Pneumonia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>